### Домашняя работа к уроку 5

Тема “POS-tagger и NER”

#### Задание 1. Написать теггер на данных с русским языком
1.	проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2.	написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3.	сравнить все реализованные методы сделать выводы
 
#### Задание 2. Проверить насколько хорошо работает NER - в отдельном notebook
данные брать из http://www.labinform.ru/pub/named_entities/
1.	проверить NER из nltk/spacy/deeppavlov
2.	написать свой нер попробовать разные подходы
a.	передаём в сетку токен и его соседей
b.	передаём в сетку только токен
c.	свой вариант
3.	сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)

#### Задание 1

1. Проверяем UnigramTagger, BigramTagger, TrigramTagger и их комбмнации

In [1]:
import pyconll

import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [2]:
full_train = pyconll.load_from_file('./ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('./ru_syntagrus-ud-dev.conllu')

In [3]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [4]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])
    
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [5]:
default_tagger = nltk.DefaultTagger('NOUN')
default_acc = default_tagger.evaluate(fdata_test)

unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate(fdata_test)

print(f'Accuracy:\nDefault Tagger: {round(default_acc, 3)},\nUnigram Tagger: {round(unigram_acc, 3)},\nBigram Tagger: {round(bigram_acc, 3)},\n'
      f'Trigram Tagger: {round(trigram_acc, 3)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 5)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 5)},\n')

Accuracy:
Default Tagger: 0.236,
Unigram Tagger: 0.877,
Bigram Tagger: 0.696,
Trigram Tagger: 0.248,
Bigram and Unigram Tagger: 0.88298,
Trigram, Bigram and Unigram Tagger: 0.88208,



In [6]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [7]:
le = LabelEncoder()

train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

print(f'Classes: {le.classes_} \nNumber of classes: {len(le.classes_)}')

Classes: ['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NO_TAG' 'NUM' 'PART'
 'PRON' 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X'] 
Number of classes: 18


#### 2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
По буквам

In [8]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.9438546827081858
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.9469045934014086
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.9487075792808277


По словам

In [9]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='word')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(871526, 99485)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.7532184140464395
(871526, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.7721918916186432
(871526, 99485)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.7534880194115863


#### Выводы:
1. Наилучший результат из отдельных таггеров показал Unigram Tagger: 0.877.
2. Наилучший результат из всех вариантов показал векторайзер по буквам TfidfVectorizer: 0.9487
3. Все векторайзеры по словам уступили даже Unigram Tagger